In [1]:
import torch
from torch import nn
from torch.nn import functional
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding
import torch.optim as optim
from torch.profiler import profile, record_function, ProfilerActivity
import numpy as np
from tqdm import tqdm
import os

from dataset.create_dataset import BookGenerator, create_test_dataset
from layers.model import Transformer, AutoregressiveWrapper
from transformers import BertTokenizer
from test_model.test_model import TestModel

import wandb
import yadisk

In [2]:
CONFIG = {
    "architecture": "Transformer", # Wandb only
    "dataset": "books", #"wikitext-103-raw-v1", # Wandb only
    "batch_size": 8,
    "embedding_size": 768,
    "max_sequence_length": 256,
    "number_of_layers": 12,
    "number_of_heads": 12,
    "additional_feed_forward_layers": 0,
    "extention_factor": 4,
    "dropout_rate": 0.1,
    'train_size': 2**20,
    'test_size': 32,
    'start_book': 0,
    'flash_atten': False,
    'num_train_books': 25000,
    'save_every': 3000,
    'use_mixed_precision': True,
    'model_path': None, # "savepoints/dulcet-serenity-218",
    
}
CONFIG["lr"] = 0.001 / np.sqrt(CONFIG["batch_size"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [3]:
def log_metrics(test_name, test_dataloader, pipeline, model_tester, train_config, CONFIG):
    pipeline.eval()
    if test_name in ['books']:
        metrics = model_tester.test_model(pipeline, test_dataloader)
        test_loss = metrics['loss']
        bleu = metrics['bleu']
        rouge1 = metrics['rouge1']
        rouge2 = metrics['rouge2']
        rougeL = metrics['rougeL']

        wandb.log({
            "test_loss_" + test_name: test_loss,
            "bleu_" + test_name: bleu,
            "rouge1_" + test_name: rouge1,
            "rouge2_" + test_name: rouge2,
            "rougeL_" + test_name: rougeL,
        })
        
    else:
        simmilarity_accuracy, simmilarity_score = model_tester.test_simmilarity(pipeline,
                                                            test_dataloader, tokenizer)
        
        wandb.log({
            "simmilarity_accuracy": simmilarity_accuracy,
            "simmilarity_score": simmilarity_score,
        })

    pipeline.train()
        

def train(CONFIG, pipeline, model, optimizer, loss_function, model_tester, wandb, dataloaders):
    train_config = {
        "test_every": 2048 // CONFIG["batch_size"],
        "log_traing_metrics_every": 256 // CONFIG["batch_size"],
    }
    
    book_generator, test_book_dataloader, cloze_dataloader = dataloaders

    train_time = 0
    
    model.train()

    batch_num = 0
    train_losses = []
    tests = ['books', 'simmilarity_score']
    test_dataloaders = [test_book_dataloader, cloze_dataloader]
    
    for i in tqdm(range(CONFIG['num_train_books']), desc="Training Progress"):
        train_dataloader = book_generator.next_book()
        for batch in train_dataloader:
            train_start = time()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            optimizer.zero_grad()

            model_output, target = pipeline(input_ids, attention_mask)
            loss = loss_function(model_output.transpose(1, 2), target)

            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            train_time += time() - train_start
            batch_num += 1
    
            if batch_num % train_config["log_traing_metrics_every"] == 0:
                wandb.log({
                    "train_loss": sum(train_losses[-train_config["log_traing_metrics_every"]:]) / train_config["log_traing_metrics_every"],
                    "train_time": train_time / train_config["log_traing_metrics_every"],
                })
                train_time = 0
            
            if batch_num % train_config["test_every"] == 0:
                test_start = time()
                for i in range(len(tests)):
                    log_metrics(tests[i], test_dataloaders[i], pipeline, model_tester, train_config, CONFIG)
                wandb.log({
                    "test_time": time() - test_start,
                })

In [4]:
def create_model(CONFIG, model_path=None):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    number_of_tokens = tokenizer.vocab_size

    y = yadisk.YaDisk(token="y0_AgAAAAA7vZKNAADLWwAAAADsK4dQ-f3fKT3hSianJggcCcKC1Mfxo8s")
    print(y.check_token())
    model = Transformer(
        embedding_size=CONFIG["embedding_size"],
        number_of_tokens=number_of_tokens,
        number_of_heads=CONFIG["number_of_heads"],
        number_of_layers=CONFIG["number_of_layers"],
        extention_factor=CONFIG["extention_factor"],
        additional_feed_forward_layers=CONFIG["additional_feed_forward_layers"],
        dropout_rate=CONFIG["dropout_rate"],
        max_sequence_length=CONFIG["max_sequence_length"],
        use_flash_att=CONFIG["flash_atten"],
    ).to(device)
    if model_path:
        model.load_state_dict(torch.load(model_path))
        
    pipeline = AutoregressiveWrapper(model).to(device)
    loss_function = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=CONFIG["lr"])
    model_tester = TestModel(tokenizer, model)

    return pipeline, model, optimizer, loss_function, model_tester, y

In [5]:
def save_model(model, wandb_run, y):
    if not os.path.exists('save_models):
        os.makedirs('save_models')
    save_dir = f"save_models/{wandb_run.name}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    saved_models = [f for f in os.listdir(save_dir) if f.startswith("model_")]
    num_saved_models = len(saved_models)

    model_filename = f"model_{num_saved_models + 1}"
    model_path = f'{save_dir}/{model_filename}'
    torch.save(model.state_dict(), model_path)
    
    try:
        y.mkdir(save_dir)
    except yadisk.exceptions.PathExistsError:
        pass
    print(model_path)
    y.upload(model_path, model_path)


In [6]:
wandb_run = wandb.init(
    project="transformer",
    tags=["long_training_testing"],
    #id="sblota90",
    resume="allow",
    config=CONFIG
)

load_path = CONFIG['model_path']

wandb: Currently logged in as: skorodumov-work (8667). Use `wandb login --relogin` to force relogin


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

book_generator = BookGenerator(batch_size=CONFIG["batch_size"],
                                max_sequence_length=CONFIG["max_sequence_length"], tokenizer=tokenizer)
test_book_dataloader = book_generator.select_test(CONFIG["test_size"])
book_generator.skip_books(CONFIG['start_book'])
cloze_dataset = create_test_dataset('data.csv')

pipeline, model, optimizer, loss_function, model_tester, y = create_model(CONFIG, CONFIG["model_path"])
num_parameters, num_trainable_parameters, memory_allocated = pipeline.count_parameters() 
print('number of parameters =', num_parameters)
print('number of trainable parameters =', num_trainable_parameters)
print('memory allocated in GB =', memory_allocated)

True
number of parameters = 131968314
number of trainable parameters = 131968314
memory allocated in GB = 0.4916202798485756


In [ ]:
train(CONFIG, pipeline, model, optimizer, loss_function, model_tester, scaler, wandb,
     [book_generator, test_book_dataloader, cloze_dataset])

In [8]:
save_model(model, wandb_run, y)

save_models/divine-totem-248/model_1


In [ ]:
"""
autocast only
bs 8, 6.9 GB
7 1:23
8 1:46
9 2:38
11 4:44

autocast with scaler
bs 8, 6.8 GB
7 1:23
8 1:46
9 2:39
11 4:44

default training
bs 8, 5.8 GB
7 1:33
8 1:58
9 2:48
11 4:44

apex scaler
bs 8, 5.8 GB

"""

In [ ]:
wandb.finish()

In [ ]:
input_text = """
When you want to travel around the city, you can take a colorful car called a
"""
num_predicted_tokens = 20
k = 3
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

start = time()
answer = pipeline.predict_next(input_text, tokenizer, num_predicted_tokens, k)
print(.time() - start)

print(answer)

In [ ]:
from random import randint

def predict_next(pipeline, input_text, num_predicted_tokens, tokenizer):
    input_tokens = tokenizer.encode(input_text, return_tensors="pt")
    input_tokens = input_tokens[:, :-1].to(device)
    
    for i in range(num_predicted_tokens):
        mask = torch.ones_like(input_tokens)

        with torch.no_grad():
            probabilities = pipeline.next_token_probabilities(input_tokens, mask)
        
        answer = probabilities.argsort(dim=-1)[:, -randint(1, 2)].unsqueeze(0)
        input_tokens = torch.cat((input_tokens, answer), dim=1)
        
    return tokenizer.decode(input_tokens[0])

In [ ]:
input_text = """
london is the capital of
"""
num_predicted_tokens = 40

answer = predict_next(pipeline, input_text, num_predicted_tokens, tokenizer)

print(answer)

In [ ]:
simmilarity_accuracy, simmilarity_score = model_tester.test_simmilarity(pipeline,
                                                            cloze_dataset, tokenizer)

In [ ]:
"""
TODO
optimize
-understand time wasting
-choose batch size


beam search

new dataset
-find new dataset for big training
-make some analysis
-avg, mean, each percentile sequence length
-find info in the net

model parameters
-max sequence length
-estimate batch size
-choose parameters for n GB VRAM
-choose model parameters for 5-6 GB model

filter model
-

new conversation dataset 
-analyse
-understand how to train and test conversation model
-try to train

metrics
-create metrics for conversation

multiple GPUs
-write code to run model on several gpus
-try to run model on vast.ai
-also model must be saved per epoch

site
-create django project with model which is placed on google colab
-create 


"""

In [9]:
33.20 * 1.15


38.18